# ４章　レポーティングする仕組みを構築する10本ノック

### ノック３１：特定店舗の売上をExcelにして出力してみよう

In [1]:
import pandas as pd
import glob
import os

m_store = pd.read_csv("m_store.csv")
m_area = pd.read_csv("m_area.csv")

current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir, "tbl_order_*.csv")
tbl_order_files = glob.glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
  order_tmp = pd.read_csv(file)
  print(f"{file}:{len(order_tmp)}")
  order_all = pd.concat([order_all, order_tmp], ignore_index=True)

# 保守等店舗のデータ削除
order_all = order_all.loc[order_all["store_id"] != 999]

order_all = pd.merge(order_all, m_store, on="store_id", how="left")
order_all = pd.merge(order_all, m_area, on="area_cd", how="left")

# マスターにないコードに対応した文字列を設定
order_all.loc[order_all["takeout_flag"]==0, "takeout_name"] = "デリバリー"
order_all.loc[order_all["takeout_flag"]==1, "takeout_name"] = "お持ち帰り"

order_all.loc[order_all["status"]==0, "status_name"] = "受付"
order_all.loc[order_all["status"]==1, "status_name"] = "お支払済"
order_all.loc[order_all["status"]==2, "status_name"] = "お渡し済"
order_all.loc[order_all["status"]==9, "status_name"] = "キャンセル"

order_all.loc[:, "order_date"] = pd.to_datetime(order_all["order_accept_date"]).dt.date

order_all.head()

/Users/graduser/Documents/GitHub/100knock/PYTHON/04章/tbl_order_202006.csv:233301
/Users/graduser/Documents/GitHub/100knock/PYTHON/04章/tbl_order_202004.csv:233260
/Users/graduser/Documents/GitHub/100knock/PYTHON/04章/tbl_order_202005.csv:241139


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_date
0,16514392,64,C44271355,49,56527968,2020-06-01 11:00:00,2020-06-01 11:55:00,0,2732,2,西東京店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
1,28342394,12,C98307866,98,46924304,2020-06-01 11:00:00,2020-06-01 11:52:00,0,2064,2,西葛西店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
2,29799587,174,C59484037,45,47054474,2020-06-01 11:00:00,2020-06-01 11:25:00,0,3900,2,新座店,SA,埼玉,埼玉,デリバリー,お渡し済,2020-06-01
3,96960753,23,C57278332,70,60848267,2020-06-01 11:00:00,2020-06-01 11:45:00,0,2750,2,江東店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
4,52147209,7,C36890849,80,20633593,2020-06-01 11:00:00,2020-06-01 11:23:00,0,2328,2,東尾久店,TK,東京,東京,デリバリー,お渡し済,2020-06-01


In [2]:
import openpyxl

wb = openpyxl.Workbook()
ws = wb["Sheet"]
ws.cell(1,1).value = "書き込みテストです"
wb.save("test.xlsx")
wb.close()

In [3]:
wb = openpyxl.load_workbook("test.xlsx", read_only=True)
ws = wb["Sheet"]
print(ws.cell(1,1).value)
wb.close()

書き込みテストです


In [4]:
# テストデータの準備
store_id = 1
store_df = order_all.loc[order_all["store_id"] == store_id].copy()
store_name = store_df["store_name"].unique()[0]
store_sales_total = store_df.loc[store_df["status"].isin([1,2])]["total_amount"].sum()
store_sales_takeout = store_df.loc[store_df["status"]==1]["total_amount"].sum()
store_sales_delivery = store_df.loc[store_df["status"]==2]["total_amount"].sum()
print(f"売上額確認 {store_sales_total} = {store_sales_takeout + store_sales_delivery}")
output_df = store_df[["order_accept_date", "customer_id", "total_amount", "takeout_name", "status_name"]]
output_df.head()

売上額確認 9004535 = 9004535


,order_accept_date,customer_id,total_amount,takeout_name,status_name
58,2020-06-01 11:05:05,C13019060,3530,デリバリー,お渡し済
222,2020-06-01 11:16:16,C42292941,3617,デリバリー,お渡し済
309,2020-06-01 11:24:24,C04466742,2877,お持ち帰り,お支払済
641,2020-06-01 11:52:52,C05244872,2987,お持ち帰り,お支払済
1118,2020-06-01 12:31:31,C27811735,1882,お持ち帰り,お支払済


In [5]:
from openpyxl.utils.dataframe import dataframe_to_rows

store_title = f"{store_id}_{store_name}"

wb = openpyxl.Workbook()
ws = wb.active
ws.title = store_title

ws.cell(1,1).value = f"{store_title} 売り上げデータ"

# OpenPyXL ののユーティリティ dataframe_to_rows を利用
rows = dataframe_to_rows(output_df, index=False, header=True)

# 表の貼り付け位置
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３２：Excelの表を整えて出力してみよう

In [6]:
# スタイル関係系のインポート
from openpyxl.styles import PatternFill, Border, Side, Font

openpyxl.load_workbook(filename)
ws = wb[store_title]

side = Side(style="thin", color="008080")
border = Border(top=side, bottom=side, left=side, right=side)

# データの表の部分に罫線を設定
for row in ws:
  for cell in row:
    if ws[cell.coordinate].value:
      ws[cell.coordinate].border = border

ws.cell(1,1).font = Font(bold=True, color="008080")

cell = ws.cell(3,2)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文受注日時"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,3)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "顧客ID"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,4)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "購入総額"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,5)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文タイプ"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3,6)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文状態"
cell.font = Font(bold=True, color="FFFFFF")

ws.column_dimensions["A"].width = 20
ws.column_dimensions["B"].width = 20
ws.column_dimensions["C"].width = 12
ws.column_dimensions["D"].width = 12
ws.column_dimensions["E"].width = 12
ws.column_dimensions["F"].width = 12

# ファイルに保存
wb.save(filename)
wb.close()


### ノック３３：売上以外のデータも出力してみよう

In [7]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

store_df.loc[:, "order_accept_datetime"] = pd.to_datetime(store_df["order_accept_date"])
store_df.loc[:, "delivered_datetime"] = pd.to_datetime(store_df["delivered_date"])
store_df.loc[:, "delta"] = store_df[["order_accept_datetime", "delivered_datetime"]].apply(calc_delta, axis=1)

delivery_time = store_df.groupby(["store_id"])["delta"].describe()
delivery_time

,count,mean,std,min,25%,50%,75%,max
store_id,,,,,,,,
1,3553.0,34.477062,14.514403,10.0,22.0,34.0,47.0,59.0


In [8]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(1,7)
cell.value = f"配達完了までの時間"
cell.font = Font(bold=True, color="008080")

rows = dataframe_to_rows(delivery_time, index=False, header=True)

# 表の貼り付け位置
row_start = 3
col_start = 8

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        cell = ws.cell(row_no, col_no)
        cell.value = value
        cell.border = border
        if row_no == row_start:
            cell.fill = PatternFill(patternType="solid", fgColor="008080")
            cell.font = Font(bold=True, color="FFFFFF")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３４：問題のある箇所を赤字で出力してみよう

In [9]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

rows = dataframe_to_rows(output_df, index=False, header=True)

# 表の貼り付け位置
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    if row_no == row_start:
        continue
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value
        if value == "キャンセル":
            ws.cell(row_no, col_no).font = Font(bold=True, color="FF0000")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３５：エクセルのセル関数で日毎の集計をしてみよう

In [10]:
openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7,7)
cell.value = "集計"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8,8)
cell.value = "データ総額"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8,10)
cell.value = f"=SUM(D4:D{ws.max_row})"

cell = ws.cell(9,8)
cell.value = " 内 決済完了額"
cell.font = Font(bold=True)

cell = ws.cell(9,10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"<>"&"キャンセル", D4:D{ws.max_row})'

cell = ws.cell(10,8)
cell.value = " 内 キャンセル額"
cell.font = Font(bold=True)

cell = ws.cell(10,10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"="&"キャンセル", D4:D{ws.max_row})'

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### ノック３６：折れ線グラフにして出力してみよう

In [11]:
from openpyxl.chart import Reference, BarChart, PieChart, LineChart, ScatterChart, Series

openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7,7)
cell.value = f'売上グラフ'
cell.font = Font(bold=True, color='008080')

# グラフ用の参照データを指定、D列（購入総額）の４行目から20件を指定
refy = Reference(ws, min_col=4, min_row=4, max_col=4, max_row=23)

# グラフシリーズを生成
series = Series(refy, title='売上額')

# Chart
chart = LineChart()
chart.title = '折れ線グラフ'
chart.x_axis.title = '件数'
chart.y_axis.title = '売上額'
chart.height = 10
chart.width = 20
chart.series.append(series)

# 生成したChartオブジェクトをシートの指定位置に追加
ws.add_chart(chart, 'H12')

filename = f'{store_title}.xlsx'
wb.save(filename)
wb.close()

### ノック３７：レポートに向けてデータを準備しよう

In [12]:
# キャンセル率ランキングデータを準備
cancel_df = pd.DataFrame()
cancel_cnt = order_all.loc[order_all['status']==9].groupby(['store_id'])['store_id'].count()
order_cnt = order_all.loc[order_all['status'].isin([1,2,9])].groupby(['store_id'])['store_id'].count()
cancel_rate = (cancel_cnt / order_cnt) * 100
cancel_df['cancel_rate'] = cancel_rate
cancel_df = pd.merge(cancel_df, m_store, on='store_id', how='left')
cancel_rank = cancel_df.sort_values('cancel_rate', ascending=True).reset_index()

def check_store_cancel_rank(trg_id):
    tmp = cancel_rank.loc[cancel_rank['store_id'] == trg_id].index + 1
    return tmp[0]

In [24]:
def get_area_df(trg_id):
    # 該当店舗が属する、地域別データの集計と売上ランク
    area_df = pd.DataFrame()
    area_df = order_all.loc[order_all['area_cd']==store_df['area_cd'].unique()[0]]
    area_df = area_df.loc[area_df['status'].isin([1,2])]
    return area_df

def get_area_rank_df(trg_id):
    area_df = get_area_df(trg_id)
    area_rank = area_df.groupby(['store_id'])['total_amount'].sum().sort_values(ascending=True)
    area_rank = pd.merge(area_rank, m_store, on='store_id', how='left')
    return area_rank

def check_store_sales_rank(trg_id):
    area_rank = get_area_rank_df(trg_id)
    tmp = area_rank.loc[area_rank['store_id']==trg_id].index + 1
    return tmp[0]

def make_store_daily(trg_id):
    # 該当店舗の日毎売上データ
    tmp_store_df = order_all.loc[(order_all['store_id']==trg_id) & (order_all['status'].isin([1,2]))]
    tmp = tmp_store_df[['order_accept_date', 'total_amount']].copy()
    tmp.loc[:, 'order_accept_date'] = pd.to_datetime(tmp['order_accept_date'])
    tmp.set_index('order_accept_date', inplace=True)
    tmp = tmp.resample('D').sum().reset_index()
    return tmp

def get_area_delivery(trg_id):
    # 該当店舗が属する、地域別データの配達完了までの時間ランク
    area_delivery = pd.DataFrame()
    area_df = get_area_df(trg_id)
    area_delivery = area_df.loc[area_df['status']==2].copy()
    area_delivery.loc[:, 'order_accept_datetime'] = pd.to_datetime(area_delivery['order_accept_date'])
    area_delivery.loc[:, 'delivered_datetime'] = pd.to_datetime(area_delivery['delivered_date'])
    area_delivery.loc[:, 'delta'] = area_delivery[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
    return area_delivery

def get_area_delivery_rank_df(trg_id):
    area_delivery = get_area_delivery(trg_id)
    area_delivery_rank = area_delivery.groupby(['store_id'])['delta'].mean().sort_values()
    area_delivery_rank = pd.merge(area_delivery_rank, m_store, on='store_id', how='left')
    return area_delivery_rank

def check_store_delivery_rank(trg_id):
    area_delivery_rank = get_area_delivery_rank_df(trg_id)
    tmp = area_delivery_rank.loc[area_delivery_rank['store_id']==trg_id].index + 1
    return tmp[0]

### ノック３８：データシートに必要なデータを出力しよう

In [25]:
# 最初にテスト用のファイルを削除
if os.path.exists('test.xlsx') : os.remove('test.xlsx')
if os.path.exists(filename) : os.remove(filename)

In [26]:
# 複数のデータをシートに貼り付けるための汎用的な関数
def data_sheet_output(trg_wb, sheetname, target_wb, indexFlg):
  ws = trg_wb.create_sheet(title=sheetname)
  rows = dataframe_to_rows(target_wb, index=indexFlg, header=True)

  # 表の貼り付け位置
  row_start = 1
  col_start = 1

  for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
      ws.cell(row_no, col_no).value = value

  # データシートは非表示にしておく
  ws.sheet_state = 'hidden'

# 上記の関数を呼び出す関数を定義
def make_data_sheet(trg_id, trg_st_df, targetfolder):
  target_daily = make_store_daily(trg_id)
  store_name = trg_st_df['store_name'].unique()[0]

  # 新たにファイルを作成する
  store_title = f'{trg_id}_{store_name}'
  wb = openpyxl.Workbook()

  # キャンセルランキング
  data_sheet_output(wb, 'Data_CancelRank', cancel_rank, False)
  # エリア売り上げランキング
  data_sheet_output(wb, 'Data_AreaRank', get_area_rank_df(trg_id), False)
  # エリア配送時間ランキング
  data_sheet_output(wb, 'Data_DeliveryRank', get_area_delivery_rank_df(trg_id), False)
  # 該当店舗の日単位売り上げデータ
  data_sheet_output(wb, 'Data_TargetRank', target_daily, False)

  filename = os.path.join(targetfolder, f'{store_title}.xlsx')
  wb.save(filename)
  wb.close()

  return filename

In [27]:
filename_store = make_data_sheet(store_id, store_df, '')

### ノック３９：サマリーシートを作成しよう

In [ ]:
def make_summary_sheet(trg_id, storename, trgfile):
  target_cancel_rank = check_store_cancel_rank(trg_id)
  target_sales_rank = check_store_sales_rank(trg_id)
  target_delivery_rank = check_store_delivery_rank(trg_id)

  wb = openpyxl.load_workbook(trgfile)
  ws = wb.activews.title = 'サマリーレポート'

  cell = ws.cell(1,1)
  cell.value = f'{storename} サマリーレポート (4月~6月)'
  cell.font = Font(bold=True, color='008080', size=20)

  # 売り上げランキングの表示
  tmpWs = wb['Data_Target_Daily']
  cell = ws.cell(3,2)
  cell.value = '店舗売上額'
  cell.font = Font(bold=True, color='008080', size=16)

  # セルの結合
  ws.merge_cells('E3:F3')

  cell = ws.cell(3,5)
  cell.value = f'=SUM({tmpWs.title}!B2:B{tmpWs.max_row})'
  cell.font = Font(bold=True, color='0080FF', size=16)
  cell.number_format = '#,##0'

  cell = ws.cell(4,2)
  cell.value = '店舗売上ランク'
  cell.font = Font(bold=True, color='008080', size=16)

  cell = ws.cell(4,5)
  cell.value = f'{len(m_store)}店舗中 {target_sales_rank}位'
  cell.font = Font(bold=True, color='0080FF', size=16)

  # グラフ用の参照データを指定
  refy = Reference(tmpWs, min_col=2, min_row=2, max_col=2, max_row=tmpWs.max_row)

  # グラフシリーズを生成
  series = Series(refy, title='売上額')

  # Chart
  chart = LineChart()
  chart.title = '期間売上額（日毎）'
  chart.x_axis.title = '件数'
  chart.y_axis.title = '売上額'
  chart.height = 10
  chart.width = 15
  chart.series.append(series)



### ノック４０：店舗別にレポートをExcel出力してみよう